In [1]:
!pip install torch torchvision transformers sentence-transformers

  Using cached torchvision-0.20.0-cp39-cp39-win_amd64.whl.metadata (6.2 kB)
  Using cached sentence_transformers-3.2.1-py3-none-any.whl.metadata (10 kB)
  Using cached torch-2.5.0-cp39-cp39-win_amd64.whl.metadata (28 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached torchvision-0.20.0-cp39-cp39-win_amd64.whl (1.6 MB)
   -------------                            67.1/203.0 MB ? eta -:--:--


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\jjj\anaconda\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\jjj\anaconda\lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\Users\jjj\anaconda\lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "C:\Users\jjj\anaconda\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
  File "C:\Users\jjj\anaconda\lib\http\client.py", line 463, in read
    n = self.readinto(b)
  File "C:\Users\jjj\anaconda\lib\http\client.py", line 507, in readinto
    n = self.fp.readinto(b)
  File "C:\Users\jjj\anaconda\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\jjj\anaconda\lib\ssl.py", line 1241, 

In [1]:
import csv

# Define the data
data = [
    ["train/Apple rust leaf/Apple rust leaf (1).jpg", "Apple Rust Leaf", "Yellow-orange spots, premature leaf fall", "Use fungicide containing myclobutanil"],
    ["train/Bell_pepper leaf spot/Bell_pepper leaf (1).jpg", "Bell Pepper Leaf Spot", "Dark, water-soaked spots on leaves", "Apply copper-based fungicide"],
    ["train/Corn rust leaf/Corn rust leaf (1).jpg", "Corn Rust Leaf", "Reddish-brown pustules, leaf wilting", "Use resistant corn varieties and fungicides"]
]

# Create the CSV file
with open('plant_diseases.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["filename", "disease_name", "symptoms", "treatment"])
    writer.writerows(data)

print("CSV file created successfully!")


CSV file created successfully!


In [5]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import pandas as pd

# Initialize CLIP model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Define paths and load sample data
dataset_path = "./dataset/train"
disease_directories = ["Apple rust leaf", "Bell_pepper leaf spot", "Corn rust leaf"]

# Create annotations for sample purposes (replace with actual data if available)
annotations = [
    {
        "filename": "Apple rust leaf/Apple rust leaf (1).jpg",
        "disease": "Apple Rust Leaf",
        "symptoms": "Yellow-orange spots, premature leaf fall",
        "treatment": "Use fungicide containing myclobutanil"
    },
    {
        "filename": "Bell_pepper leaf spot//Bell_pepper leaf (1).jpg",
        "disease": "Bell Pepper Leaf Spot",
        "symptoms": "Dark, water-soaked spots on leaves",
        "treatment": "Apply copper-based fungicide"
    },
    {
        "filename": "Corn rust leaf/Corn rust leaf (1).jpg",
        "disease": "Corn Rust Leaf",
        "symptoms": "Reddish-brown pustules, leaf wilting",
        "treatment": "Use resistant corn varieties and fungicides"
    }
]

# Function to encode images using CLIP
def encode_image(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)
    return image_features

# Function to encode text using CLIP
def encode_text(text):
    inputs = processor(text=text, return_tensors="pt").to(device)
    with torch.no_grad():
        text_features = model.get_text_features(**inputs)
    return text_features

# Function to match user query and provide a response
def get_disease_info(image_path, query):
    image_features = encode_image(image_path)
    text_features = encode_text(query)

    # Simulate matching process
    highest_similarity = float('-inf')
    best_match = None

    for annotation in annotations:
        disease_text = f"{annotation['disease']}: {annotation['symptoms']}"
        disease_features = encode_text(disease_text)

        # Cosine similarity
        similarity = torch.nn.functional.cosine_similarity(text_features, disease_features, dim=1).item()

        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = annotation

    # Provide response based on the best match
    if best_match:
        print(f"Disease: {best_match['disease']}")
        print(f"Symptoms: {best_match['symptoms']}")
        print(f"Treatment: {best_match['treatment']}")
    else:
        print("No relevant information found.")

# Example usage
image_path = "./PlantDoc-Dataset/train/Apple rust leaf/Apple rust leaf (2).jpg"
user_query = "How to treat yellowing leaves on my plant?"
get_disease_info(image_path, user_query)


Disease: Bell Pepper Leaf Spot
Symptoms: Dark, water-soaked spots on leaves
Treatment: Apply copper-based fungicide


In [8]:
pip install torch torchvision torchaudio

   ---------------------------------------- 2.4/2.4 MB 4.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Device configuration
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define paths and directories
dataset_path = "./PlantDoc-dataset/train"
disease_directories = ["Apple rust leaf", "Bell_pepper leaf spot", "Corn rust leaf"]

# Annotation data for creating CSV
annotations = [
    {
        "filename": "Apple rust leaf/Apple rust leaf (2).jpg",
        "disease": "Apple Rust Leaf",
        "symptoms": "Yellow-orange spots, premature leaf fall",
        "treatment": "Use fungicide containing myclobutanil"
    },
    {
        "filename": "Bell_pepper leaf spot/Bell_pepper leaf spot (3).jpg",
        "disease": "Bell Pepper Leaf Spot",
        "symptoms": "Dark, water-soaked spots on leaves",
        "treatment": "Apply copper-based fungicide"
    },
    {
        "filename": "Corn rust leaf/Corn rust leaf (5).jpg",
        "disease": "Corn Rust Leaf",
        "symptoms": "Reddish-brown pustules, leaf wilting",
        "treatment": "Use resistant corn varieties and fungicides"
    }
]

# Create CSV from annotations
annotations_df = pd.DataFrame(annotations)
annotations_df.to_csv("disease_annotations.csv", index=False)

# Create dictionary for symptoms and treatments
disease_info = {
    "Apple Rust Leaf": {
        "symptoms": "Yellow-orange spots, premature leaf fall",
        "treatment": "Use fungicide containing myclobutanil"
    },
    "Bell Pepper Leaf Spot": {
        "symptoms": "Dark, water-soaked spots on leaves",
        "treatment": "Apply copper-based fungicide"
    },
    "Corn Rust Leaf": {
        "symptoms": "Reddish-brown pustules, leaf wilting",
        "treatment": "Use resistant corn varieties and fungicides"
    }
}

# Define custom dataset class
class PlantDiseaseDataset(Dataset):
    def __init__(self, root_dir, disease_dirs, transform=None):
        self.image_paths = []
        self.labels = []
        self.transform = transform
        # Loop over disease directories
        for idx, disease_dir in enumerate(disease_dirs):
            disease_path = os.path.join(root_dir, disease_dir)
            for img_name in os.listdir(disease_path):
                img_path = os.path.join(disease_path, img_name)
                self.image_paths.append(img_path)
                self.labels.append(idx)  # Assign a numeric label for each disease
    
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Initialize dataset and dataloaders
dataset = PlantDiseaseDataset(dataset_path, disease_directories, transform=transform)
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
    
    def forward(self, x):
        return self.model(x)

# Initialize model, criterion, and optimizer
num_classes = len(disease_directories)
cnn_model = SimpleCNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# Training loop (only 2 epochs for demonstration)
for epoch in range(2):
    cnn_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        # Forward pass
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/2], Loss: {loss.item():.4f}")

# Save the trained model
torch.save(cnn_model.state_dict(), "plant_disease_cnn.pth")

# Load CLIP model and processor for integration
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Function to encode images using CLIP
def encode_image_with_clip(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = clip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs)
    return image_features

# Function to encode text using CLIP
def encode_text_with_clip(text):
    inputs = clip_processor(text=text, return_tensors="pt").to(device)
    with torch.no_grad():
        text_features = clip_model.get_text_features(**inputs)
    return text_features

# Function to integrate CNN and CLIP
def integrated_diagnosis(image_path, text_query):
    # Use CNN to classify image
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0).to(device)
    cnn_model.eval()
    with torch.no_grad():
        cnn_output = cnn_model(image_tensor)
        _, predicted_class = torch.max(cnn_output, 1)
        disease_name = disease_directories[predicted_class.item()]

    # Fetch symptoms and treatment
    disease_key = disease_name.replace("_", " ").title()
    symptoms = disease_info[disease_key]["symptoms"]
    treatment = disease_info[disease_key]["treatment"]
    print(f"CNN Prediction: {disease_name}")
    print(f"Symptoms: {symptoms}")
    print(f"Treatment: {treatment}")
    
    # Use CLIP to enhance diagnosis
    image_features = encode_image_with_clip(image_path)
    text_features = encode_text_with_clip(text_query)
    similarity = torch.nn.functional.cosine_similarity(image_features, text_features, dim=1).item()
    print(f"CLIP Similarity with Query: {similarity}")

# Example usage
integrated_diagnosis("./PlantDoc-dataset/train/Apple rust leaf/Apple rust leaf (13).jpg", "How to treat yellowing spots on apple leaves?")


C:\Users\jjj\anaconda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\jjj\anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/2], Loss: 1.6213
Epoch [2/2], Loss: 1.1660
CNN Prediction: Apple rust leaf
Symptoms: Yellow-orange spots, premature leaf fall
Treatment: Use fungicide containing myclobutanil
CLIP Similarity with Query: 0.28555938601493835


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Define fine-tuning parameters directly in the Python code
config = {
    "epochs": 15,
    "batch_size": 16,
    "learning_rate": 0.0001,
    "clip_lr": 0.00005,
    "cnn_model": "resnet18",
    "fine_tune_layers": ["layer4", "fc"]
}

# Device configuration
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define paths and directories
dataset_path = "./PlantDoc-dataset/train"
disease_directories = ["Apple rust leaf", "Bell_pepper leaf spot", "Corn rust leaf"]

# Annotation data for creating CSV
annotations = [
    {
        "filename": "Apple rust leaf/Apple rust leaf (2).jpg",
        "disease": "Apple Rust Leaf",
        "symptoms": "Yellow-orange spots, premature leaf fall",
        "treatment": "Use fungicide containing myclobutanil"
    },
    {
        "filename": "Bell_pepper leaf spot/Bell_pepper leaf spot (3).jpg",
        "disease": "Bell Pepper Leaf Spot",
        "symptoms": "Dark, water-soaked spots on leaves",
        "treatment": "Apply copper-based fungicide"
    },
    {
        "filename": "Corn rust leaf/Corn rust leaf (5).jpg",
        "disease": "Corn Rust Leaf",
        "symptoms": "Reddish-brown pustules, leaf wilting",
        "treatment": "Use resistant corn varieties and fungicides"
    }
]

# Create CSV from annotations
annotations_df = pd.DataFrame(annotations)
annotations_df.to_csv("disease_annotations.csv", index=False)

# Create dictionary for symptoms and treatments
disease_info = {
    "Apple Rust Leaf": {
        "symptoms": "Yellow-orange spots, premature leaf fall",
        "treatment": "Use fungicide containing myclobutanil"
    },
    "Bell Pepper Leaf Spot": {
        "symptoms": "Dark, water-soaked spots on leaves",
        "treatment": "Apply copper-based fungicide"
    },
    "Corn Rust Leaf": {
        "symptoms": "Reddish-brown pustules, leaf wilting",
        "treatment": "Use resistant corn varieties and fungicides"
    }
}

# Define custom dataset class
class PlantDiseaseDataset(Dataset):
    def __init__(self, root_dir, disease_dirs, transform=None):
        self.image_paths = []
        self.labels = []
        self.transform = transform
        # Loop over disease directories
        for idx, disease_dir in enumerate(disease_dirs):
            disease_path = os.path.join(root_dir, disease_dir)
            for img_name in os.listdir(disease_path):
                img_path = os.path.join(disease_path, img_name)
                self.image_paths.append(img_path)
                self.labels.append(idx)  # Assign a numeric label for each disease

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Initialize dataset and dataloaders
dataset = PlantDiseaseDataset(dataset_path, disease_directories, transform=transform)
train_loader = DataLoader(dataset, batch_size=config["batch_size"], shuffle=True)

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.model = models.resnet18(pretrained=True)
        # Freeze all layers except specified layers for fine-tuning
        for name, param in self.model.named_parameters():
            param.requires_grad = False
            if any(layer in name for layer in config["fine_tune_layers"]):
                param.requires_grad = True
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Initialize model, criterion, and optimizer
num_classes = len(disease_directories)
cnn_model = SimpleCNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=config["learning_rate"])

# Training loop with fine-tuning
for epoch in range(config["epochs"]):
    cnn_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        # Forward pass
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{config['epochs']}], Loss: {loss.item():.4f}")

# Save the trained model
torch.save(cnn_model.state_dict(), "plant_disease_cnn_finetuned.pth")


C:\Users\jjj\anaconda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\jjj\anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/15], Loss: 0.5983
Epoch [2/15], Loss: 0.0113
Epoch [3/15], Loss: 0.0824
Epoch [4/15], Loss: 0.0686
Epoch [5/15], Loss: 0.1040
Epoch [6/15], Loss: 0.0102
Epoch [7/15], Loss: 0.3106
Epoch [8/15], Loss: 0.0401
Epoch [9/15], Loss: 0.0048
Epoch [10/15], Loss: 0.0041
Epoch [11/15], Loss: 0.1992
Epoch [12/15], Loss: 0.0043
Epoch [13/15], Loss: 0.0855
Epoch [14/15], Loss: 0.0014
Epoch [15/15], Loss: 0.0255


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import pandas as pd
import json

# JSON configuration (embedded directly within the code)
config_json = """
{
    "device": "cuda",
    "dataset_path": "./PlantDoc-dataset/train",
    "disease_directories": ["Apple rust leaf", "Bell_pepper leaf spot", "Corn rust leaf"],
    "annotations": [
        {
            "filename": "Apple rust leaf",
            "disease": "Apple Rust Leaf",
            "symptoms": "Yellow-orange spots, premature leaf fall",
            "treatment": "Use fungicide containing myclobutanil"
        },
        {
            "filename": "Bell_pepper leaf spot",
            "disease": "Bell Pepper Leaf Spot",
            "symptoms": "Dark, water-soaked spots on leaves",
            "treatment": "Apply copper-based fungicide"
        },
        {
            "filename": "Corn rust leaf",
            "disease": "Corn Rust Leaf",
            "symptoms": "Reddish-brown pustules, leaf wilting",
            "treatment": "Use resistant corn varieties and fungicides"
        }
    ],
    "train_batch_size": 8,
    "num_epochs": 10,
    "learning_rate": 0.001
}
"""

# Parse JSON configuration
config = json.loads(config_json)
device = config["device"] if torch.cuda.is_available() else "cpu"
dataset_path = config["dataset_path"]
disease_directories = config["disease_directories"]

# Create CSV from annotations
annotations_df = pd.DataFrame(config["annotations"])
annotations_df.to_csv("disease_annotations.csv", index=False)

# Create dictionary for symptoms and treatments
disease_info = {
    annotation["disease"]: {
        "symptoms": annotation["symptoms"],
        "treatment": annotation["treatment"]
    }
    for annotation in config["annotations"]
}

# Define custom dataset class
class PlantDiseaseDataset(Dataset):
    def __init__(self, root_dir, disease_dirs, transform=None):
        self.image_paths = []
        self.labels = []
        self.transform = transform
        # Loop over disease directories
        for idx, disease_dir in enumerate(disease_dirs):
            disease_path = os.path.join(root_dir, disease_dir)
            for img_name in os.listdir(disease_path):
                img_path = os.path.join(disease_path, img_name)
                self.image_paths.append(img_path)
                self.labels.append(idx)  # Assign a numeric label for each disease

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Initialize dataset and dataloaders
dataset = PlantDiseaseDataset(dataset_path, disease_directories, transform=transform)
train_loader = DataLoader(dataset, batch_size=config["train_batch_size"], shuffle=True)

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Initialize model, criterion, and optimizer
num_classes = len(disease_directories)
cnn_model = SimpleCNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=config["learning_rate"])

# Training loop (configurable number of epochs)
for epoch in range(config["num_epochs"]):
    cnn_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        # Forward pass
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch + 1}/{config['num_epochs']}], Loss: {loss.item():.4f}")

# Save the trained model
torch.save(cnn_model.state_dict(), "plant_disease_cnn.pth")

# Load CLIP model and processor for integration
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Function to encode images using CLIP
def encode_image_with_clip(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = clip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs)
    return image_features

# Function to encode text using CLIP
def encode_text_with_clip(text):
    inputs = clip_processor(text=text, return_tensors="pt").to(device)
    with torch.no_grad():
        text_features = clip_model.get_text_features(**inputs)
    return text_features

# Function to integrate CNN and CLIP
def integrated_diagnosis(image_path, text_query):
    # Use CNN to classify image
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0).to(device)
    cnn_model.eval()
    with torch.no_grad():
        cnn_output = cnn_model(image_tensor)
        _, predicted_class = torch.max(cnn_output, 1)
        disease_name = disease_directories[predicted_class.item()]
    
    # Fetch symptoms and treatment
    disease_key = disease_name.replace("_", " ").title()
    symptoms = disease_info[disease_key]["symptoms"]
    treatment = disease_info[disease_key]["treatment"]
    print(f"CNN Prediction: {disease_name}")
    print(f"Symptoms: {symptoms}")
    print(f"Treatment: {treatment}")

    # Use CLIP to enhance diagnosis
    image_features = encode_image_with_clip(image_path)
    text_features = encode_text_with_clip(text_query)
    similarity = torch.nn.functional.cosine_similarity(image_features, text_features, dim=1).item()
    print(f"CLIP Similarity with Query: {similarity}")

# Example usage
integrated_diagnosis("./PlantDoc-dataset/train/Apple rust leaf/Apple rust leaf (13).jpg", "How to treat yellowing spots on apple leaves?")


C:\Users\jjj\anaconda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\jjj\anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 0.9199
Epoch [2/10], Loss: 0.4417
Epoch [3/10], Loss: 0.2912
Epoch [4/10], Loss: 1.3282
Epoch [5/10], Loss: 0.1439
Epoch [6/10], Loss: 0.1904
Epoch [7/10], Loss: 0.3712
Epoch [8/10], Loss: 0.5786
Epoch [9/10], Loss: 0.5023
Epoch [10/10], Loss: 0.1858
CNN Prediction: Apple rust leaf
Symptoms: Yellow-orange spots, premature leaf fall
Treatment: Use fungicide containing myclobutanil
CLIP Similarity with Query: 0.28555938601493835


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import pandas as pd
import json
from sklearn.model_selection import train_test_split

# JSON configuration (embedded directly within the code)
config_json = """
{
    "device": "cuda",
    "dataset_path": "./PlantDoc-dataset/train",
    "disease_directories": ["Apple rust leaf", "Bell_pepper leaf spot", "Corn rust leaf"],
    "annotations": [
        {
            "filename": "Apple rust leaf",
            "disease": "Apple Rust Leaf",
            "symptoms": "Yellow-orange spots, premature leaf fall",
            "treatment": "Use fungicide containing myclobutanil"
        },
        {
            "filename": "Bell_pepper leaf spot",
            "disease": "Bell Pepper Leaf Spot",
            "symptoms": "Dark, water-soaked spots on leaves",
            "treatment": "Apply copper-based fungicide"
        },
        {
            "filename": "Corn rust leaf",
            "disease": "Corn Rust Leaf",
            "symptoms": "Reddish-brown pustules, leaf wilting",
            "treatment": "Use resistant corn varieties and fungicides"
        }
    ],
    "train_batch_size": 8,
    "num_epochs": 10,
    "learning_rate": 0.001
}
"""

# Parse JSON configuration
config = json.loads(config_json)
device = config["device"] if torch.cuda.is_available() else "cpu"
dataset_path = config["dataset_path"]
disease_directories = config["disease_directories"]

# Create CSV from annotations
annotations_df = pd.DataFrame(config["annotations"])
annotations_df.to_csv("disease_annotations.csv", index=False)

# Create dictionary for symptoms and treatments
disease_info = {
    annotation["disease"]: {
        "symptoms": annotation["symptoms"],
        "treatment": annotation["treatment"]
    }
    for annotation in config["annotations"]
}

# Define custom dataset class
class PlantDiseaseDataset(Dataset):
    def __init__(self, root_dir, disease_dirs, transform=None):
        self.image_paths = []
        self.labels = []
        self.transform = transform
        # Loop over disease directories
        for idx, disease_dir in enumerate(disease_dirs):
            disease_path = os.path.join(root_dir, disease_dir)
            for img_name in os.listdir(disease_path):
                img_path = os.path.join(disease_path, img_name)
                self.image_paths.append(img_path)
                self.labels.append(idx)  # Assign a numeric label for each disease

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Initialize dataset and split into train/validation sets
dataset = PlantDiseaseDataset(dataset_path, disease_directories, transform=transform)
train_paths, val_paths = train_test_split(dataset.image_paths, test_size=0.2, random_state=42)
train_dataset = PlantDiseaseDataset(dataset_path, disease_directories, transform=transform)
val_dataset = PlantDiseaseDataset(dataset_path, disease_directories, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=config["train_batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config["train_batch_size"], shuffle=False)

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Initialize model, criterion, and optimizer
num_classes = len(disease_directories)
cnn_model = SimpleCNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=config["learning_rate"])

# Function to calculate accuracy
def calculate_accuracy(model, dataloader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total * 100
    return accuracy

# Training loop
for epoch in range(config["num_epochs"]):
    cnn_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        # Forward pass
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # After each epoch, calculate the training accuracy
    train_accuracy = calculate_accuracy(cnn_model, train_loader)
    val_accuracy = calculate_accuracy(cnn_model, val_loader)
    print(f"Epoch [{epoch+1}/{config['num_epochs']}], Loss: {loss.item():.4f}, Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")

# Save the trained model
torch.save(cnn_model.state_dict(), "plant_disease_cnn.pth")

# Load CLIP model and processor for integration
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Function to encode images using CLIP
def encode_image_with_clip(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = clip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs)
    return image_features

# Function to encode text using CLIP
def encode_text_with_clip(text):
    inputs = clip_processor(text=text, return_tensors="pt").to(device)
    with torch.no_grad():
        text_features = clip_model.get_text_features(**inputs)
    return text_features

# Function to integrate CNN and CLIP
def integrated_diagnosis(image_path, text_query):
    # Use CNN to classify image
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0).to(device)
    cnn_model.eval()
    with torch.no_grad():
        cnn_output = cnn_model(image_tensor)
        _, predicted_class = torch.max(cnn_output, 1)
        disease_name = disease_directories[predicted_class.item()]
        # Fetch symptoms and treatment
    disease_key = disease_name.replace("_", " ").title()
    symptoms = disease_info[disease_key]["symptoms"]
    treatment = disease_info[disease_key]["treatment"]
    print(f"CNN Prediction: {disease_name}")
    print(f"Symptoms: {symptoms}")
    print(f"Treatment: {treatment}")
    # Use CLIP to enhance diagnosis
    image_features = encode_image_with_clip(image_path)
    text_features = encode_text_with_clip(text_query)
    similarity = torch.nn.functional.cosine_similarity(image_features, text_features, dim=1).item()
    print(f"CLIP Similarity with Query: {similarity}")

# query 1
integrated_diagnosis("./PlantDoc-dataset/train/Apple rust leaf/Apple rust leaf (13).jpg", 
                     "How to treat yellowing spots on apple leaves?")


C:\Users\jjj\anaconda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\jjj\anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 1.1598, Training Accuracy: 60.73%, Validation Accuracy: 60.73%
Epoch [2/10], Loss: 0.4354, Training Accuracy: 66.80%, Validation Accuracy: 66.80%
Epoch [3/10], Loss: 0.6718, Training Accuracy: 92.71%, Validation Accuracy: 92.71%
Epoch [4/10], Loss: 0.5094, Training Accuracy: 81.38%, Validation Accuracy: 81.38%
Epoch [5/10], Loss: 0.7085, Training Accuracy: 85.83%, Validation Accuracy: 85.83%
Epoch [6/10], Loss: 0.0596, Training Accuracy: 85.43%, Validation Accuracy: 85.43%
Epoch [7/10], Loss: 0.1723, Training Accuracy: 93.93%, Validation Accuracy: 93.93%
Epoch [8/10], Loss: 1.3988, Training Accuracy: 83.40%, Validation Accuracy: 83.40%
Epoch [9/10], Loss: 0.1616, Training Accuracy: 87.45%, Validation Accuracy: 87.45%
Epoch [10/10], Loss: 0.2520, Training Accuracy: 95.14%, Validation Accuracy: 95.14%
CNN Prediction: Apple rust leaf
Symptoms: Yellow-orange spots, premature leaf fall
Treatment: Use fungicide containing myclobutanil
CLIP Similarity with Query: 0.2855593

In [8]:
# query 2

integrated_diagnosis("./PlantDoc-dataset/train/Bell_pepper leaf spot/Bell_pepper leaf spot (7).jpg", 
                     "what are the dark spots on the leaves?")

CNN Prediction: Bell_pepper leaf spot
Symptoms: Dark, water-soaked spots on leaves
Treatment: Apply copper-based fungicide
CLIP Similarity with Query: 0.281999796628952


In [10]:
# query 3

integrated_diagnosis("./PlantDoc-dataset/train/Corn rust leaf/Corn rust leaf (6).jpg", 
                     "why is that reddish brown color?")

CNN Prediction: Corn rust leaf
Symptoms: Reddish-brown pustules, leaf wilting
Treatment: Use resistant corn varieties and fungicides
CLIP Similarity with Query: 0.2102934718132019


In [12]:
# query 4

integrated_diagnosis("./PlantDoc-dataset/train/Corn rust leaf/Corn rust leaf (6).jpg", 
                     "how to treat this leaf wilting?")

CNN Prediction: Corn rust leaf
Symptoms: Reddish-brown pustules, leaf wilting
Treatment: Use resistant corn varieties and fungicides
CLIP Similarity with Query: 0.23080112040042877
